<a href="https://colab.research.google.com/github/wojiushilr/ML_hw/blob/master/4_RNN/hw4_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!cp "/content/drive/My Drive/kaggle/hw4/ml2020spring-hw4.zip" /content
!cp "/content/drive/My Drive/kaggle/hw4/w2v_all.model" /content

In [ ]:
!ls
!unzip ml2020spring-hw4.zip

In [6]:
!ls

drive		      sample_data	training_label.txt    w2v_all.model
ml2020spring-hw4.zip  testing_data.txt	training_nolabel.txt



### 這個 block 用來先定義一些等等常用到的函式


In [7]:
def load_training_data(path='training_label.txt'):
    if "training_label" in path:
        with open(path, 'r') as f:
            lines = f.readlines()
            lines = [line.strip('\n').split(' ') for line in lines]
            print(lines[0])
        x = [line[2:] for line in lines]
        y = [line[0] for line in lines]
        # print(x[0])
        # print(y[0])
        return x, y
    else:
        with open(path, 'r') as f:
            lines = f.readlines()
            x = [line.strip('\n').split(' ') for line in lines]
        return x

def load_testing_data(path='testing_data'):
    # 把 testing 時需要的 data 讀進來
    with open(path, 'r') as f:
        lines = f.readlines()
        X = ["".join(line.strip('\n').split(",")[1:]).strip() for line in lines[1:]]
        X = [sen.split(' ') for sen in X]
        print(X[0])
    return X

def evaluation(outputs, labels):
    # outputs => probability (float)
    # labels => labels
    outputs[outputs>=0.5] = 1 # 大於等於 0.5 為正面
    outputs[outputs<0.5] = 0 # 小於 0.5 為負面
    correct = torch.sum(torch.eq(outputs, labels)).item()
    return correct

### 這個 block 是用來訓練 word to vector 的 word embedding
### 预计embedding耗时10min

In [ ]:
# '''
# 读取数据
# w2v.py
# 這個 block 是用來訓練 word to vector 的 word embedding
# '''
# import os
# import numpy as np
# import pandas as pd
# import argparse
# from gensim.models import Word2Vec
# def train_word2vec(x):
#     # 訓練 word to vector 的 word embedding
#     model = Word2Vec(x, size=250, window=5, min_count=5, workers=12, iter=10, sg=1)
#     return model
# if __name__ == "__main__":
#     print("loading training data ...")
#     train_x, y = load_training_data('training_label.txt')
#     train_x_no_label = load_training_data('training_nolabel.txt')

#     print("loading testing data ...")
#     test_x = load_testing_data('testing_data.txt')

#     # model = train_word2vec(train_x + train_x_no_label + test_x)
#     model = train_word2vec(train_x + test_x)

#     print("saving model ...")
#     # model.save(os.path.join(path_prefix, 'model/w2v_all.model'))
#     model.save(os.path.join('w2v_all.model'))

### 数据处理
### NLP类似问题可以重复使用

In [8]:
class Preprocess():
    def __init__(self, sentences, sen_len, w2v_path="./w2v.model"):
        self.w2v_path = w2v_path
        self.sentences = sentences
        self.sen_len = sen_len
        self.idx2word = []
        self.word2idx = {}
        self.embedding_matrix = []
    def get_w2v_model(self):
        # 把之前訓練好的 word to vec 模型讀進來
        self.embedding = Word2Vec.load(self.w2v_path)
        self.embedding_dim = self.embedding.vector_size
    def add_embedding(self, word):
        # 把 word 加進 embedding，並賦予他一個隨機生成的 representation vector
        vector = torch.empty(1, self.embedding_dim)
        torch.nn.init.uniform_(vector) #  均匀分布
        self.word2idx[word] = len(self.word2idx)
        self.idx2word.append(word)
        self.embedding_matrix = torch.cat([self.embedding_matrix, vector], 0) # 指定维度为0，所以按行拼接

    def make_embedding(self, load=True):
        print("Get embedding ...")
        # 取得訓練好的 Word2vec word embedding
        if load:
            print("loading word to vec model ...")
            self.get_w2v_model()
        else:
            raise NotImplementedError
        # 製作一個 word2idx 的 dictionary
        # 製作一個 idx2word 的 list
        # 製作一個 word2vector 的 list
        for i, word in enumerate(self.embedding.wv.vocab):
            print('get words #{}'.format(i + 1), end='\r')
            # e.g. self.word2index['he'] = 1
            # e.g. self.index2word[1] = 'he'
            # e.g. self.vectors[1] = 'he' vector
            self.word2idx[word] = len(self.word2idx)
            self.idx2word.append(word)
            self.embedding_matrix.append(self.embedding[word])
        print('')
        self.embedding_matrix = torch.tensor(self.embedding_matrix)
        # 將 "<PAD>" 跟 "<UNK>" 加進 embedding 裡面
        self.add_embedding("<PAD>")
        self.add_embedding("<UNK>")
        print("total words: {}".format(len(self.embedding_matrix)))
        return self.embedding_matrix

    def pad_sequence(self, sentence):
        # 將每個句子變成一樣的長度
        if len(sentence) > self.sen_len:
            sentence = sentence[:self.sen_len]
        else:
            pad_len = self.sen_len - len(sentence)
            for _ in range(pad_len):
                sentence.append(self.word2idx["<PAD>"])
        assert len(sentence) == self.sen_len
        return sentence

    def sentence_word2idx(self):
        # 把句子裡面的字轉成相對應的 index
        sentence_list = []
        for i, sen in enumerate(self.sentences):
            print('sentence count #{}'.format(i + 1), end='\r')
            sentence_idx = []
            for word in sen:
                if (word in self.word2idx.keys()):
                    sentence_idx.append(self.word2idx[word])
                else:
                    sentence_idx.append(self.word2idx["<UNK>"])
            # 將每個句子變成一樣的長度
            sentence_idx = self.pad_sequence(sentence_idx)
            sentence_list.append(sentence_idx)
        return torch.LongTensor(sentence_list)

    def labels_to_tensor(self, y):
        # 把 labels 轉成 tensor
        y = [int(label) for label in y]
        return torch.LongTensor(y)

### 重写 dataset 所需要的 '__init__', '__getitem__', '__len__'

In [9]:
'''
data.py
dataset 所需要的 '__init__', '__getitem__', '__len__'
之后配合 dataloader 使用
'''
import torch
from torch.utils.data import Dataset
class myDataset(Dataset):
    def __init__(self,X,y):
        self.data = X
        self.label = y
    def __getitem__(self, idx):
        if self.label == None:
            return  self.data[idx]
        else:
            return  self.data[idx], self.label[idx]
    def __len__(self):
        return len(self.data)

### 模型定义

In [22]:
'''
model.py
模型定义
'''
import torch
from torch import nn
class myLSTM(nn.Module):
    def __init__(self, embedding, embedding_dim, hidden_dim, num_layers, dropout=0.5, fix_embedding=True):
        super(myLSTM,self).__init__()
        # 初始化 embedding lookup表，
        self.embed = torch.nn.Embedding(embedding.size(0),embedding.size(1)) # 0 -> 词数，1 -> 向量的大小
        # 给上纪的表赋予之前处理得到的embedding，不给weight赋值的话就是随即值
        self.embed.weight = torch.nn.Parameter(embedding)
        # 是否將 embedding fix 住，如果 fix_embedding 為 False，在訓練過程中，embedding 也會跟著被訓練
        if fix_embedding:
            self.embed.weight.requires_grad = False
        else:
            self.embed.weight.requires_grad = True
        # self.embed_dim = embedding.size(1)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.classifier = nn.Sequential(nn.Dropout(dropout),
                                        nn.Linear(hidden_dim, 1),
                                        nn.Sigmoid())
    # 定义forward流程
    def forward(self, inputs):
        inputs = self.embed(inputs)
        ##  此处不在意cn，hn所以为None
        x, _ = self.lstm(inputs, None)
        # 因为batch_first=True，所以 x 的 dimension (batch, seq_len, hidden_size)
        # 取用 LSTM 最後一層的 hidden state
        x = x[:, -1, :]
        x = self.classifier(x)
        return x

### 训练


In [24]:
'''
train.py
'''
def training(batch_size, n_epoch, lr, model_dir, train, valid, model, device):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print('\nstart training, parameter total:{}, trainable:{}\n'.format(total, trainable))
    criterion = nn.BCELoss() # 定義損失函數，這裡我們使用 binary cross entropy loss
    t_batch = len(train)
    v_batch = len(valid)
    optimizer = optim.Adam(model.parameters(), lr=lr)  # 將模型的參數給 optimizer，並給予適當的 learning rate
    total_loss, total_acc, best_acc = 0, 0, 0
    for epoch in range(n_epoch):
        model.train()
        total_loss, total_acc = 0, 0
        # 這段做 training
        for i, (inputs, labels) in enumerate(train):
            inputs = inputs.to(device, dtype=torch.long)  # device 為 "cuda"，將 inputs 轉成 torch.cuda.LongTensor
            labels = labels.to(device, dtype=torch.float)  # 將 labels 轉成 torch.cuda.FloatTensor，因為等等要餵進 criterion，所以型態要是 float
            optimizer.zero_grad()  # 由於 loss.backward() 的 gradient 會累加，所以每次餵完一個 batch 後需要歸零
            outputs = model(inputs)  # 將 input 餵給模型
            outputs = outputs.squeeze()  # 去掉最外面的 dimension，好讓 outputs 可以餵進 criterion(), 比如10 classes，output为(10,1),要把它转换为(10,)
            loss = criterion(outputs, labels)  # 計算此時模型的 training loss
            loss.backward()  # 算 loss 的 gradient
            optimizer.step()  # 更新訓練模型的參數
            correct = evaluation(outputs, labels)  # 計算此時模型的 training accuracy
            total_acc += (correct / batch_size)
            total_loss += loss.item()
            # print('[ Epoch{}: {}/{} ] loss:{:.3f} acc:{:.3f} '.format(
            #     epoch + 1, i + 1, t_batch, loss.item(), correct * 100 / batch_size), end='\r')
        print('\nTrain | Loss:{:.5f} Acc: {:.3f}'.format(total_loss / t_batch, total_acc / t_batch * 100))

        # 這段做 validation
        model.eval()  # 將 model 的模式設為 eval，這樣 model 的參數就會固定住
        with torch.no_grad():
            total_loss, total_acc = 0, 0
            for i, (inputs, labels) in enumerate(valid):
                inputs = inputs.to(device, dtype=torch.long)
                labels = labels.to(device, dtype=torch.float)
                outputs = model(inputs)  # 將 input 餵給模型
                outputs = outputs.squeeze()
                loss = criterion(outputs, labels) # 計算此時模型的 validation loss
                correct = evaluation(outputs, labels)
                total_acc += (correct / batch_size)
                total_loss += loss.item()
            print("Valid | Loss:{:.5f} Acc: {:.3f} ".format(total_loss / v_batch, total_acc / v_batch * 100))
            if total_acc > best_acc:
                # 如果 validation 的結果優於之前所有的結果，就把當下的模型存下來以備之後做預測時使用
                best_acc = total_acc
                # torch.save(model, "{}/val_acc_{:.3f}.model".format(model_dir,total_acc/v_batch*100))
                torch.save(model, "{}/ckpt.model".format(model_dir))
                print('saving model with acc {:.3f}'.format(total_acc / v_batch * 100))
        print('-----------------------------------------------')

### 测试

In [12]:
'''
test.py
'''
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F

def testing(batch_size, test_loader, model, device):
    model.eval()
    ret_output = []
    with torch.no_grad():
        for i, inputs in enumerate(test_loader):
            inputs = inputs.to(device, dtype=torch.long)
            outputs = model(inputs)
            outputs = outputs.squeeze()
            outputs[outputs>=0.5] = 1 # 大於等於 0.5 為正面
            outputs[outputs<0.5] = 0 # 小於 0.5 為負面
            ret_output += outputs.int().tolist()

    return ret_output

### 主程序


In [29]:
'''
main.py
主程序
'''
import os
import torch
import argparse
import numpy as np
import pandas as pd
from torch import nn
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split

# 通過 torch.cuda.is_available() 的回傳值進行判斷是否有使用 GPU 的環境，如果有的話 device 就設為 "cuda"，沒有的話就設為 "cpu"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
path_prefix ='./'

# 處理好各個 data 的路徑
train_with_label = os.path.join(path_prefix, 'training_label.txt')
train_no_label = os.path.join(path_prefix, 'training_nolabel.txt')
testing_data = os.path.join(path_prefix, 'testing_data.txt')

w2v_path = os.path.join('w2v_all.model') # 處理 word to vec model 的路徑

# 定义句子长度、是否固定 embedding、batch 大小、epoch、learning rate 的值、model 的資料夾路徑等参数
sen_len = 20
fix_embedding = True # fix embedding during training
batch_size = 128
epoch = 5
lr = 0.001

model_dir = path_prefix # model directory for checkpoint model

print("loading training data ...") # 把 'training_label.txt' 跟 'training_nolabel.txt' 讀進來
train_x, y = load_training_data(train_with_label)
train_x_no_label = load_training_data(train_no_label)

# 对于input和label做预处理
preprocess = Preprocess(train_x, sen_len, w2v_path=w2v_path)
embedding = preprocess.make_embedding(load=True)
train_x = preprocess.sentence_word2idx()
y = preprocess.labels_to_tensor(y)

# 製作一個 model 的對象
model = myLSTM(embedding, embedding_dim=250, hidden_dim=150, num_layers=1, dropout=0.5, fix_embedding=fix_embedding)
model = model.to(device) # device為 "cuda"，model 使用 GPU 來訓練（餵進去的 inputs 也需要是 cuda tensor）

# 把 data 分為 training data 跟 validation data（將一部份 training data 拿去當作 validation data）
X_train, X_val, y_train, y_val = train_x[:180000], train_x[180000:], y[:180000], y[180000:]

# 把 data 做成 dataset 供 dataloader 取用
train_dataset = myDataset(X=X_train, y=y_train)
val_dataset =myDataset(X=X_val, y=y_val)
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                            batch_size = batch_size,
                                            shuffle = True,
                                            num_workers = 8)
val_loader = torch.utils.data.DataLoader(dataset = val_dataset,
                                            batch_size = batch_size,
                                            shuffle = False,
                                            num_workers = 8)
# 開始訓練
training(batch_size, epoch, lr, model_dir, train_loader, val_loader, model, device)


# 开始测试
print("loading testing data ...")
test_x = load_testing_data(testing_data)

# 对于input和label做预处理
preprocess = Preprocess(test_x, sen_len, w2v_path=w2v_path)
embedding = preprocess.make_embedding(load=True)
test_x = preprocess.sentence_word2idx()
test_dataset = myDataset(X=test_x, y=None)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                            batch_size = batch_size,
                                            shuffle = False,
                                            num_workers = 8)
print('\nload model ...')
model = torch.load(os.path.join(model_dir, 'ckpt.model'))
outputs = testing(batch_size, test_loader, model, device)

# 寫到 csv 檔案供上傳 Kaggle
tmp = pd.DataFrame({"id":[str(i) for i in range(len(test_x))],"label":outputs})
print("save csv ...")
tmp.to_csv(os.path.join(path_prefix, 'predict.csv'), index=False)
print("Finish Predicting")

loading training data ...
['1', '+++$+++', 'are', 'wtf', '...', 'awww', 'thanks', '!']
Get embedding ...
loading word to vec model ...


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


get words #2204

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


get words #24694
total words: 24696

start training, parameter total:6415351, trainable:241351


Train | Loss:0.49742 Acc: 74.914
Valid | Loss:0.46239 Acc: 78.269 
saving model with acc 78.269
-----------------------------------------------


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type myLSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "



Train | Loss:0.44254 Acc: 79.131
Valid | Loss:0.44040 Acc: 79.354 
saving model with acc 79.354
-----------------------------------------------

Train | Loss:0.42684 Acc: 80.177
Valid | Loss:0.42818 Acc: 79.683 
saving model with acc 79.683
-----------------------------------------------

Train | Loss:0.41364 Acc: 80.925
Valid | Loss:0.42962 Acc: 79.777 
saving model with acc 79.777
-----------------------------------------------

Train | Loss:0.40237 Acc: 81.562
Valid | Loss:0.42101 Acc: 80.215 
saving model with acc 80.215
-----------------------------------------------
loading testing data ...
['my', 'dog', 'ate', 'our', 'dinner', '.', 'no', '', 'seriously', '...', 'he', 'ate', 'it', '.']
Get embedding ...
loading word to vec model ...
get words #24694
total words: 24696
sentence count #200000
load model ...
save csv ...
Finish Predicting
